In [1]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark 3.5.0
!wget -q https://apache.osuosl.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

In [2]:
ls -l # check the .tgz is there

total 392496
-rw-r--r-- 1 root root   1512362 Jan 17 08:53 healthcare_dataset.csv
drwxr-xr-x 1 root root      4096 Jan 12 19:20 sample_data/
-rw-r--r-- 1 root root 400395283 Sep  9 02:10 spark-3.5.0-bin-hadoop3.tgz


In [3]:
# unzip it
!tar xf spark-3.5.0-bin-hadoop3.tgz

In [4]:
!pip install -q findspark

In [5]:
!pip install py4j

# For maps
!pip install folium
!pip install plotly

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

In [7]:
import findspark
findspark.init("spark-3.5.0-bin-hadoop3")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .appName("Aggregations and Grouping") \
        .master("local[*]") \
        .getOrCreate()

spark.version

'3.5.0'

In [8]:
spark

In [9]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [20]:
# Import sql functions
from pyspark.sql.functions import *
from pyspark.sql.window import Window

# Contexto del análisis

El presente análisis de datos sobre ingresos hospitalarios en diferentes hospitales busca obtener información para la eficiencia operativa y la planificación financiera de los hospitales. Al identificar las enfermedades más costosas y comprender la demografía de los pacientes, los hospitales pueden optimizar la asignación de recursos, mejorar la atención al paciente y tomar decisiones estratégicas para adaptarse a las necesidades específicas de la comunidad atendida.

Este análisis no solo se centra en aspectos económicos, sino que también aborda la dinámica demográfica y epidemiológica de los pacientes facilitando la gestión hospitalaria.

In [11]:
#mostrar 10 lineas de la tabla y ver el esquema
health= spark.read.option("header","true").option("delimiter",",").csv("/content/healthcare_dataset.csv")
health.show(10)
health.printSchema()

+-------------------+---+------+----------+-----------------+-----------------+---------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+
|               Name|Age|Gender|Blood Type|Medical Condition|Date of Admission|         Doctor|            Hospital|Insurance Provider|    Billing Amount|Room Number|Admission Type|Discharge Date| Medication|Test Results|
+-------------------+---+------+----------+-----------------+-----------------+---------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+
|    Tiffany Ramirez| 81|Female|        O-|         Diabetes|       2022-11-17| Patrick Parker|    Wallace-Hamilton|          Medicare| 37490.98336352819|        146|      Elective|    2022-12-01|    Aspirin|Inconclusive|
|        Ruben Burns| 35|  Male|        O+|           Asthma|       2023-06-01|  Diane Jackson|Burke, Griffin an

In [12]:
#agrupar por hospital y analizar para cada hospital: el total de lo que han facturado, por cada visita la media de facturacion y el numero de visitas que han habido
facturacion_por_hospital = health.groupBy(health.Hospital).agg((sum(col("Billing Amount")).alias("facturacion total")),(avg(col("Billing Amount")).alias("media de faturacion por visita")),(count(col("Name")).alias("numero de visitas"))).orderBy((col("facturacion total")),ascending=False)
facturacion_por_hospital.show(15)

+--------------+------------------+------------------------------+-----------------+
|      Hospital| facturacion total|media de faturacion por visita|numero de visitas|
+--------------+------------------+------------------------------+-----------------+
|Smith and Sons| 477638.8811239098|            28096.404771994694|               17|
|     Smith PLC|  432283.548553998|            22751.765713368317|               19|
|     Smith Ltd|428163.07306176703|            30583.076647269074|               14|
|     Smith Inc|351463.88837185566|            25104.563455132546|               14|
|  Williams LLC| 327522.4726443278|            27293.539387027315|               12|
|  Thomas Group| 327045.3745053134|             29731.39768230122|               11|
|   Johnson Ltd| 305971.1924701309|            27815.562951830085|               11|
|Davis and Sons|293452.52035344584|            32605.835594827317|                9|
|     Jones Inc| 290351.1938483233|             32261.24376092481

In [13]:
#valorar que enfermedad supone mayores costes de tratar
enfermedades_mas_costes = health.groupBy(col("Medical Condition")).agg((avg(col("Billing Amount"))).alias("coste medio de tratar esa enfermedad")).orderBy((col("coste medio de tratar esa enfermedad")),ascending=False)
enfermedades_mas_costes.show()

+-----------------+------------------------------------+
|Medical Condition|coste medio de tratar esa enfermedad|
+-----------------+------------------------------------+
|         Diabetes|                  26060.116129178103|
|          Obesity|                   25720.84268283536|
|           Cancer|                  25539.096132609557|
|           Asthma|                   25416.86989527562|
|     Hypertension|                      25198.03397278|
|        Arthritis|                  25187.631255184588|
+-----------------+------------------------------------+



In [43]:
#determinar que 3 casos son los que han supuesto mayores gastos a las compañias de seguros medicos
window_hosp_enf=Window.partitionBy("Insurance Provider").orderBy((col("Billing Amount")).desc())
casos_hosp_enf = health.withColumn("mayores gastos",row_number().over(window_hosp_enf)).filter(col("mayores gastos") <= 3)
casos_hosp_enf.show(30)

+-------------------+---+------+----------+-----------------+-----------------+--------------------+--------------------+------------------+-----------------+-----------+--------------+--------------+-----------+------------+--------------+
|               Name|Age|Gender|Blood Type|Medical Condition|Date of Admission|              Doctor|            Hospital|Insurance Provider|   Billing Amount|Room Number|Admission Type|Discharge Date| Medication|Test Results|mayores gastos|
+-------------------+---+------+----------+-----------------+-----------------+--------------------+--------------------+------------------+-----------------+-----------+--------------+--------------+-----------+------------+--------------+
|      Joshua Rogers| 50|Female|        O-|          Obesity|       2022-09-17|       Thomas Hanson|      Ferguson-Smith|             Aetna|9990.811232952174|        243|        Urgent|    2022-09-18| Penicillin|Inconclusive|             1|
|    Vanessa Johnson| 73|  Male|    

In [37]:
#determinar que enfermedades se tratan más en cada hospital
#Calcular el total de casos por hospital y enfermedad
casos_hosp_enf = health.groupBy("Hospital", "Medical Condition").agg(count("Medical Condition").alias("TotalCasos")).orderBy((col("TotalCasos")),ascending=False)
casos_hosp_enf.show()


+--------------+-----------------+----------+
|      Hospital|Medical Condition|TotalCasos|
+--------------+-----------------+----------+
|     Smith PLC|        Arthritis|         8|
|Smith and Sons|         Diabetes|         6|
|     Smith PLC|     Hypertension|         5|
| Johnson Group|           Cancer|         4|
|Williams Group|          Obesity|         4|
|  Thomas Group|         Diabetes|         4|
|Robinson Group|         Diabetes|         4|
|  Williams LLC|          Obesity|         4|
|Davis and Sons|        Arthritis|         4|
|  Williams Ltd|     Hypertension|         4|
|Smith and Sons|           Cancer|         4|
|     Allen PLC|           Cancer|         4|
|   Jones Group|           Asthma|         4|
| Johnson Group|        Arthritis|         4|
| Hernandez Ltd|           Cancer|         4|
|     Smith Ltd|          Obesity|         4|
|   Johnson PLC|           Asthma|         4|
|     Smith Ltd|           Cancer|         4|
|Brown and Sons|           Cancer|

In [16]:
#analizar que tipo de enfermedades son las que predominan en los jóvenes (considerando jovenes a personas menores de 30 años)
jovenes_ingresados= health.filter(col("Age") <=30).groupBy(col("Medical Condition")).count().orderBy((col("count")),ascending=False)
jovenes_ingresados.show()

+-----------------+-----+
|Medical Condition|count|
+-----------------+-----+
|     Hypertension|  344|
|           Asthma|  323|
|           Cancer|  317|
|          Obesity|  315|
|         Diabetes|  304|
|        Arthritis|  300|
+-----------------+-----+



In [17]:
#analizar que tipo de enfermedades son las que predominan en los mayores (considerando mayores a personas mayores de 65 años)
jovenes_ingresados= health.filter(col("Age") >=65).groupBy(col("Medical Condition")).count().orderBy((col("count")),ascending=False)
jovenes_ingresados.show()

+-----------------+-----+
|Medical Condition|count|
+-----------------+-----+
|           Cancer|  526|
|           Asthma|  519|
|         Diabetes|  510|
|        Arthritis|  510|
|          Obesity|  505|
|     Hypertension|  501|
+-----------------+-----+



In [19]:
#determinar los dias que ha estado ingresado un paciente y ver que enfermedades implica estar más dias
from pyspark.sql.types import DateType
from datetime import datetime

#transformar el tipo de dato
health =health.withColumn("Date of Admission", col("Date of Admission").cast(DateType()))
health =health.withColumn("Discharge Date", col("Discharge Date").cast(DateType()))

# Calcular la diferencia de dias que hay entre las fechas
health_dias_ingresado=health.withColumn("dias ingresado",datediff( col("Discharge Date"),col("Date of Admission")))
health_dias_ingresado.show()

#enfermedades que suponen mayores dias de ingreso
hospital_dias_ingresado=health_dias_ingresado.groupBy(col("Medical Condition")).agg((avg(col("dias ingresado"))).alias("dias medios ingresado")).orderBy((col("dias medios ingresado")),ascending=False)
hospital_dias_ingresado.show()


+--------------------+---+------+----------+-----------------+-----------------+------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+--------------+
|                Name|Age|Gender|Blood Type|Medical Condition|Date of Admission|            Doctor|            Hospital|Insurance Provider|    Billing Amount|Room Number|Admission Type|Discharge Date| Medication|Test Results|dias ingresado|
+--------------------+---+------+----------+-----------------+-----------------+------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+--------------+
|     Tiffany Ramirez| 81|Female|        O-|         Diabetes|       2022-11-17|    Patrick Parker|    Wallace-Hamilton|          Medicare| 37490.98336352819|        146|      Elective|    2022-12-01|    Aspirin|Inconclusive|            14|
|         Ruben Burns| 35|  Male|   